# Mezclando estilos diferentes de programación usando helpers

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

[(x_train, y_train), (x_test, y_test)]=tf.keras.datasets.cifar10.load_data ()

In [2]:
num_classes=10
y_train = tf.keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = tf.keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [3]:
print('x train:',x_train.shape)
x_train=x_train/255
x_test=x_test/255
#pd.get_dummies(y_train)
print('y train:',y_train.shape)
num_labels=10
print('x test:',x_test.shape)
print('y test:',y_test.shape)


x train: (50000, 32, 32, 3)
y train: (50000, 10)
x test: (10000, 32, 32, 3)
y test: (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 16
image_size= 32
patch_size=5
depth = 32
depth2=64
redu=8
num_hidden = 4096
num_hidden_2=1024
num_channels=3


# helpers


In [6]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#funcion helper convulucion con stride 1 y padding same
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#Max pooling stride 2, kernel 2 y padding same
def max_pool_2x2(x):
    return tf.nn.max_pool(x,[1,2,2,1],[1,2,2,1],padding='SAME')

# conv layer

def conv_layer(input,shape):
    W=weight_variable(shape)
    b=bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input,W)+b)
#full layer 
def full_layer(input,size):
    in_size=int(input.get_shape()[1])
    W=weight_variable([in_size,size])
    b=bias_variable([size])
    return tf.matmul(input,W)+b

In [7]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(x_test,dtype=tf.float32)
  keep_prob=tf.placeholder(tf.float32)  

  def model(data,keep_prob):
        conv1=conv_layer(data,shape=[patch_size,patch_size,num_channels,depth])
        conv1_pool=max_pool_2x2(conv1)
        conv2=conv_layer(conv1_pool,shape=[patch_size,patch_size,depth,depth2])
        conv2_pool=max_pool_2x2(conv2)
        conv2_flat=tf.reshape(conv2_pool,[-1,redu*redu*depth2])
        #fully connected
        full_1=tf.nn.relu(full_layer(conv2_flat,num_hidden_2))
        full_1_drop=tf.nn.dropout(full_1,keep_prob=keep_prob)
        return full_layer(full_1_drop,10)
  
  logits = model(tf_train_dataset,keep_prob=0.5) 
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
  learning_rate=1e-3
  optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset,keep_prob=1.0))  

In [31]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = x_train[offset:(offset + batch_size), :, :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
     
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
Minibatch loss at step 0: 22.775911
Minibatch accuracy: 6.2%
Minibatch loss at step 50: 2.263086
Minibatch accuracy: 31.2%
Minibatch loss at step 100: 1.899450
Minibatch accuracy: 25.0%
Minibatch loss at step 150: 2.140332
Minibatch accuracy: 25.0%
Minibatch loss at step 200: 1.976562
Minibatch accuracy: 37.5%
Minibatch loss at step 250: 2.259469
Minibatch accuracy: 18.8%
Minibatch loss at step 300: 1.446151
Minibatch accuracy: 62.5%
Minibatch loss at step 350: 1.388528
Minibatch accuracy: 37.5%
Minibatch loss at step 400: 1.543568
Minibatch accuracy: 37.5%
Minibatch loss at step 450: 1.826297
Minibatch accuracy: 31.2%
Minibatch loss at step 500: 1.968922
Minibatch accuracy: 12.5%
Minibatch loss at step 550: 2.017456
Minibatch accuracy: 31.2%
Minibatch loss at step 600: 1.659150
Minibatch accuracy: 56.2%
Minibatch loss at step 650: 1.925183
Minibatch accuracy: 25.0%
Minibatch loss at step 700: 1.609171
Minibatch accuracy: 31.2%
Minibatch loss at step 750: 1.949434
Minibatch

# modelo diferente

In [15]:
patch_size=3
batch_size=32
C1,C2,C3=30,50,80
F1=500
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(x_test,dtype=tf.float32)
  keep_prob=tf.placeholder(tf.float32)  
  def model(data,keep_prob):
        
        
        conv1_1=conv_layer(data,shape=[patch_size,patch_size,num_channels,C1])
        conv1_2=conv_layer(conv1_1,shape=[patch_size,patch_size,C1,C1])
        conv1_3=conv_layer(conv1_2,shape=[patch_size,patch_size,C1,C1])
        conv1_pool=max_pool_2x2(conv1_3)
        conv1_drop=tf.nn.dropout(conv1_pool,keep_prob)
     
        conv2_1=conv_layer(conv1_drop,shape=[patch_size,patch_size,C1,C2])
        conv2_2=conv_layer(conv2_1,shape=[patch_size,patch_size,C2,C2])
        conv2_3=conv_layer(conv2_2,shape=[patch_size,patch_size,C2,C2])
        conv2_pool=max_pool_2x2(conv2_3)
        conv2_drop=tf.nn.dropout(conv2_pool,keep_prob)    
        
        conv3_1=conv_layer(conv2_drop,shape=[patch_size,patch_size,C2,C3])
        conv3_2=conv_layer(conv3_1,shape=[patch_size,patch_size,C3,C3])
        conv3_3=conv_layer(conv3_2,shape=[patch_size,patch_size,C3,C3])
        conv3_pool=tf.nn.max_pool(conv3_3,[1,8,8,1],[1,8,8,1],padding='SAME')
        
        #flat
        conv3_flat=tf.reshape(conv3_pool,[-1,C3])
        conv3_drop=tf.nn.dropout(conv3_flat,keep_prob)

        #fully connected
        full_1=tf.nn.relu(full_layer(conv3_drop,F1))
        full_1_drop=tf.nn.dropout(full_1,keep_prob=keep_prob)
        return full_layer(full_1_drop,10)
  
  logits = model(tf_train_dataset,keep_prob=0.5) 
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
  learning_rate=1e-3
  optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset,keep_prob=1.0))  


In [16]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = x_train[offset:(offset + batch_size), :, :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
     
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
Minibatch loss at step 0: 67.822128
Minibatch accuracy: 0.0%
Minibatch loss at step 50: 2.335329
Minibatch accuracy: 12.5%
Minibatch loss at step 100: 2.381514
Minibatch accuracy: 6.2%
Minibatch loss at step 150: 2.260598
Minibatch accuracy: 18.8%
Minibatch loss at step 200: 2.410365
Minibatch accuracy: 12.5%
Minibatch loss at step 250: 2.279494
Minibatch accuracy: 9.4%
Minibatch loss at step 300: 2.299127
Minibatch accuracy: 15.6%
Minibatch loss at step 350: 2.290463
Minibatch accuracy: 3.1%
Minibatch loss at step 400: 2.297115
Minibatch accuracy: 9.4%
Minibatch loss at step 450: 2.304297
Minibatch accuracy: 15.6%
Minibatch loss at step 500: 2.318113
Minibatch accuracy: 3.1%
Minibatch loss at step 550: 2.284433
Minibatch accuracy: 12.5%
Minibatch loss at step 600: 2.287393
Minibatch accuracy: 12.5%
Minibatch loss at step 650: 2.322190
Minibatch accuracy: 12.5%
Minibatch loss at step 700: 2.312058
Minibatch accuracy: 6.2%
Minibatch loss at step 750: 2.273959
Minibatch accur